In [3]:
from app.db.engine import engine
from sqlalchemy import text
from IPython.display import display, HTML
import logging

# Désactiver les logs SQLAlchemy
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)

# Pokémon ID pour lequel on veut voir l'évolution
pokemon_id = 555 

# Requête pour obtenir la lignée d'évolution avec pokemon_details
query = text("""
SELECT 
    p.id, 
    p.name_en, 
    p.name_fr,
    ps.front_default
FROM 
    pokemons p
LEFT JOIN
    pokemon_sprites ps ON p.id = ps.pokemon_id
WHERE 
    p.id IN (
        -- Trouver les Pokémon dans la même chaîne d'évolution
        SELECT p1.id
        FROM pokemons p1
        JOIN pokemon_details pd1 ON p1.id = pd1.pokemon_id
        WHERE pd1.evolution_chain_id = (
            SELECT pd2.evolution_chain_id
            FROM pokemon_details pd2
            WHERE pd2.pokemon_id = :pokemon_id
        )
    )
ORDER BY 
    p.id
""")

# Exécution de la requête
with engine.connect("V2_PKMN.db") as session:
    result = session.execute(query, {"pokemon_id": pokemon_id})
    pokemons = result.fetchall()

# Affichage visuel avec sprites
html = '<div style="display: flex; justify-content: space-around; align-items: flex-end;">'

for pokemon in pokemons:
    pokemon_id, name_en, name_fr, sprite_url = pokemon
    
    # Utiliser l'URL du sprite de la base de données, ou un sprite par défaut si non disponible
    if not sprite_url:
        sprite_url = f"https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/{pokemon_id}.png"
    
    html += f'''
    <div style="text-align: center; margin: 10px;">
        <img src="{sprite_url}" style="width: 120px; height: 120px;">
        <div><b>{name_en}</b></div>
        <div>{name_fr}</div>
    </div>
    '''

html += '</div>'

# Afficher la chaîne d'évolution
display(HTML(html))

In [2]:
from app.db.engine import engine
from sqlalchemy import text
from IPython.display import display, HTML
import logging

# Désactiver les logs SQLAlchemy
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)

# Requête pour obtenir les 6 Pokémon les plus grands avec leurs détails
query = text("""
SELECT 
    p.id, 
    p.name_en, 
    p.name_fr,
    pd.weight_kg,
    pd.height_m,
    t1.name AS type_1,
    t2.name AS type_2,
    ps.official_artwork
FROM 
    pokemons p
JOIN
    pokemon_details pd ON p.id = pd.pokemon_id
JOIN
    pokemon_sprites ps ON p.id = ps.pokemon_id
JOIN
    types t1 ON p.type_1_id = t1.id
LEFT JOIN
    types t2 ON p.type_2_id = t2.id
ORDER BY 
    pd.height_m DESC
LIMIT 6
""")

# Exécution de la requête
with engine.connect("V2_PKMN.db") as session:
    result = session.execute(query)
    pokemons = result.fetchall()

# Affichage visuel des 6 Pokémon les plus grands
html = '<div style="display: flex; flex-wrap: wrap; justify-content: space-around;">'

for pokemon in pokemons:
    pokemon_id, name_en, name_fr, weight, height, type1, type2, artwork = pokemon
    
    # Utiliser l'URL de l'official artwork, ou un placeholder si non disponible
    if not artwork:
        artwork = f"https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/{pokemon_id}.png"
    
    # Afficher le second type s'il existe
    type_text = type1
    if type2:
        type_text += f" / {type2}"
    
    # Conversion des unités pour un affichage plus lisible (poids en kg, taille en m)
    weight_kg = weight / 10  # Le poids est stocké en hectogrammes (10 = 1 kg)
    height_m = height / 10   # La taille est stockée en décimètres (10 = 1 m)
    
    html += f'''
    <div style="text-align: center; margin: 15px; width: 30%;">
        <img src="{artwork}" style="width: 200px; height: 200px;">
        <div style="font-size: 1.2em; font-weight: bold;">{name_en}</div>
        <div>{name_fr}</div>
        <div style="margin-top: 5px;"><b>Taille:</b> {height_m} m</div>
        <div><b>Type:</b> {type_text}</div>
    </div>
    '''

html += '</div>'

# Afficher les Pokémon les plus grands
display(HTML(html))

In [3]:
from app.db.engine import engine
from sqlalchemy import text
import polars as pl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from IPython.display import display, HTML, Image
import logging
import matplotlib
matplotlib.use("Agg")  # Nécessaire pour la génération de GIF

# Désactiver les logs SQLAlchemy
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)

# Requête pour obtenir les statistiques moyennes par type
query = text("""
SELECT 
    t.name AS type_name,
    AVG(ps.hp) AS avg_hp,
    AVG(ps.attack) AS avg_attack,
    AVG(ps.defense) AS avg_defense,
    AVG(ps.special_attack) AS avg_special_attack,
    AVG(ps.special_defense) AS avg_special_defense,
    AVG(ps.speed) AS avg_speed
FROM 
    types t
LEFT JOIN
    pokemons p ON t.id = p.type_1_id OR t.id = p.type_2_id
LEFT JOIN
    pokemon_stats ps ON p.id = ps.pokemon_id
GROUP BY 
    t.name
HAVING 
    COUNT(p.id) > 0
ORDER BY 
    t.name
""")

# Exécution de la requête et conversion en Polars DataFrame
with engine.connect("V2_PKMN.db") as session:
    result = session.execute(query)
    data = result.fetchall()
    
    # Créer le DataFrame Polars directement
    df = pl.DataFrame(
        data,
        schema=['type_name', 'avg_hp', 'avg_attack', 'avg_defense', 
                'avg_special_attack', 'avg_special_defense', 'avg_speed']
    )

# Conversion en liste pour faciliter l'accès aux données
type_names = df['type_name'].to_list()
stats_data = df.select(['avg_hp', 'avg_attack', 'avg_defense', 'avg_special_attack', 'avg_special_defense', 'avg_speed']).to_numpy()

# Définition des couleurs pour chaque type (utiliser les noms exacts comme dans la base de données)
# Avant l'animation, affichons les types détectés pour vérifier
print("Types détectés dans la base de données:", type_names)

# Actuellement les couleurs sont définies avec la première lettre en majuscule,
# mais les types dans la BDD peuvent avoir un format différent
# Modifions pour accepter différents formats de casse
type_colors_lower = {k.lower(): v for k, v in {
    'normal': '#A8A77A', 'fire': '#EE8130', 'water': '#6390F0', 
    'electric': '#F7D02C', 'grass': '#7AC74C', 'ice': '#96D9D6', 
    'fighting': '#C22E28', 'poison': '#A33EA1', 'ground': '#E2BF65', 
    'flying': '#A98FF3', 'psychic': '#F95587', 'bug': '#A6B91A', 
    'rock': '#B6A136', 'ghost': '#735797', 'dragon': '#6F35FC', 
    'dark': '#705746', 'steel': '#B7B7CE', 'fairy': '#D685AD'
}.items()}

# Création de la figure avec un style plus moderne
plt.style.use('ggplot')
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, polar=True)

# Catégories pour le radar chart
categories = ['HP', 'Attack', 'Defense', 'Sp. Attack', 'Sp. Defense', 'Speed']
N = len(categories)

# Calcul des angles pour le radar chart
angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
angles += angles[:1]  # Fermer le graphique

# Création d'un titre pour indiquer le type courant
title = fig.suptitle('', fontsize=18, fontweight='bold', y=0.95)

def get_color_for_type(type_name):
    """Fonction auxiliaire pour obtenir la couleur d'un type, indépendamment de la casse"""
    return type_colors_lower.get(type_name.lower(), '#777777')

def update(frame):
    ax.clear()
    
    # Sélectionner le type actuel en utilisant les listes
    type_name = type_names[frame]
    color = get_color_for_type(type_name)
    
    print(f"Frame {frame}: Type={type_name}, Couleur={color}")  # Débogage
    
    # Préparer les données
    values = stats_data[frame].tolist()
    values += values[:1]  # Fermer le graphique
    
    # Dessiner le radar
    ax.plot(angles, values, color=color, linewidth=3)
    ax.fill(angles, values, color=color, alpha=0.3)
    
    # Ajouter une grille et des cercles concentriques
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=12, fontweight='bold')
    ax.grid(True, color='gray', alpha=0.3)
    
    # Définir les limites
    ax.set_ylim(0, 120)
    
    # Mettre à jour le titre avec le type et sa couleur
    title.set_text(f'Statistiques moyennes du type {type_name}')
    title.set_color(color)
    
    # Ajouter des lignes de référence sous forme de cercles concentriques
    ax.set_rticks([30, 60, 90, 120])
    ax.set_yticklabels(['30', '60', '90', '120'], fontsize=8)
    
    return ax,

# Création de l'animation
ani = FuncAnimation(fig, update, frames=len(df), interval=2000, blit=False)

# Sauvegarder l'animation en GIF
gif_path = 'pokemon_type_stats.gif'
ani.save(gif_path, writer='pillow', fps=0.5, dpi=100)

# Afficher le GIF dans le notebook
display(Image(url=gif_path))

# Afficher aussi l'animation interactive
plt.close()  # Éviter le double affichage
display(HTML(ani.to_jshtml()))

Types détectés dans la base de données: ['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']
Frame 0: Type=bug, Couleur=#A6B91A
Frame 0: Type=bug, Couleur=#A6B91A
Frame 1: Type=dark, Couleur=#705746
Frame 2: Type=dragon, Couleur=#6F35FC
Frame 3: Type=electric, Couleur=#F7D02C
Frame 4: Type=fairy, Couleur=#D685AD
Frame 5: Type=fighting, Couleur=#C22E28
Frame 6: Type=fire, Couleur=#EE8130
Frame 7: Type=flying, Couleur=#A98FF3
Frame 8: Type=ghost, Couleur=#735797
Frame 9: Type=grass, Couleur=#7AC74C
Frame 10: Type=ground, Couleur=#E2BF65
Frame 11: Type=ice, Couleur=#96D9D6
Frame 12: Type=normal, Couleur=#A8A77A
Frame 13: Type=poison, Couleur=#A33EA1
Frame 14: Type=psychic, Couleur=#F95587
Frame 15: Type=rock, Couleur=#B6A136
Frame 16: Type=steel, Couleur=#B7B7CE
Frame 17: Type=water, Couleur=#6390F0


Frame 0: Type=bug, Couleur=#A6B91A
Frame 0: Type=bug, Couleur=#A6B91A
Frame 1: Type=dark, Couleur=#705746
Frame 2: Type=dragon, Couleur=#6F35FC
Frame 3: Type=electric, Couleur=#F7D02C
Frame 4: Type=fairy, Couleur=#D685AD
Frame 5: Type=fighting, Couleur=#C22E28
Frame 6: Type=fire, Couleur=#EE8130
Frame 7: Type=flying, Couleur=#A98FF3
Frame 8: Type=ghost, Couleur=#735797
Frame 9: Type=grass, Couleur=#7AC74C
Frame 10: Type=ground, Couleur=#E2BF65
Frame 11: Type=ice, Couleur=#96D9D6
Frame 12: Type=normal, Couleur=#A8A77A
Frame 13: Type=poison, Couleur=#A33EA1
Frame 14: Type=psychic, Couleur=#F95587
Frame 15: Type=rock, Couleur=#B6A136
Frame 16: Type=steel, Couleur=#B7B7CE
Frame 17: Type=water, Couleur=#6390F0


In [12]:
from sqlalchemy import text
from app.db.engine import engine
from IPython.display import HTML

# Couleurs officielles des types Pokémon
TYPE_COLORS = {
    "electric": "#F7D02C",
    "ground": "#E2BF65",
    "flying": "#A98FF3",
}

# Requête pour récupérer les multiplicateurs
query = text("""
SELECT 
    t_def.name AS defending_type,
    te.effectiveness
FROM 
    go_types_effectiveness te
JOIN
    types t_atk ON te.attacking_type_id = t_atk.id
JOIN
    types t_def ON te.defending_type_id = t_def.id
WHERE 
    LOWER(t_atk.name) = 'ground' AND
    (LOWER(t_def.name) = 'electric' OR LOWER(t_def.name) = 'flying')
""")

# Exécution et récupération des valeurs
with engine.connect("V2_PKMN.db") as session:
    result = session.execute(query)
    type_effects = result.fetchall()

# Extraction des multiplicateurs
multipliers = {}
for defending_type, effectiveness in type_effects:
    multipliers[defending_type.lower()] = effectiveness

# Calcul du multiplicateur combiné
electric_mult = multipliers.get('electric', 1.0)
flying_mult = multipliers.get('flying', 1.0)
combined_mult = electric_mult * flying_mult

# Définir les couleurs de types
ground_color = TYPE_COLORS["ground"] 
electric_color = TYPE_COLORS["electric"]
flying_color = TYPE_COLORS["flying"]

# Créer un dégradé pour le type combiné (technique CSS)
combined_gradient = f"linear-gradient(135deg, {electric_color} 0%, {electric_color} 50%, {flying_color} 50%, {flying_color} 100%)"

# Textes d'efficacité
def get_effect_text(mult):
    if mult > 1.5:
        return "Super efficace"
    elif mult < 0.625:
        return "Pas très efficace"
    elif mult < 0.4:
        return "Très peu efficace"
    else:
        return "Efficacité normale"

# Fonction pour déterminer si le texte doit être clair ou foncé sur un fond coloré
def get_text_color(bg_color):
    # Simplifié: si la couleur est plutôt claire, renvoyer du texte foncé, sinon du texte clair
    return "#000" if bg_color in [TYPE_COLORS["electric"], TYPE_COLORS["ground"]] else "#fff"

# Affichage avec encadrés colorés aux couleurs des types
html = f"""
<div style="font-family: Arial, sans-serif; padding: 10px;">
    <h3 style="margin-bottom: 15px;">Efficacité de l'attaque Sol contre Vol/Électrik</h3>
    
    <div style="display: flex; flex-direction: column; gap: 15px;">
        <div style="display: flex; gap: 15px;">
            <div style="border-radius: 8px; padding: 12px; background-color: {ground_color}; color: {get_text_color(ground_color)}; width: 100px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); text-align: center;">
                <div style="font-weight: bold; margin-bottom: 2px;">SOL</div>
            </div>
            <div style="font-weight: bold; display: flex; align-items: center;">VS</div>
        </div>
        
        <div style="display: flex; gap: 15px; margin-bottom: 15px;">
            <div style="border-radius: 8px; padding: 12px; background-color: {electric_color}; color: {get_text_color(electric_color)}; width: 180px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
                <div style="font-weight: bold; margin-bottom: 5px;">ÉLECTRIK</div>
                <div style="font-size: 20px; font-weight: bold;">×{electric_mult}</div>
                <div>{get_effect_text(electric_mult)}</div>
            </div>
            
            <div style="border-radius: 8px; padding: 12px; background-color: {flying_color}; color: {get_text_color(flying_color)}; width: 180px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
                <div style="font-weight: bold; margin-bottom: 5px;">VOL</div>
                <div style="font-size: 20px; font-weight: bold;">×{flying_mult}</div>
                <div>{get_effect_text(flying_mult)}</div>
            </div>
        </div>
        
        <div style="border-radius: 8px; padding: 15px; background: {combined_gradient}; color: #000; width: 380px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-weight: bold; margin-bottom: 5px;">ÉLECTRIK/VOL</div>
            <div style="font-size: 24px; font-weight: bold;">×{combined_mult:.3f}</div>
            <div>{get_effect_text(combined_mult)}</div>
            <div style="margin-top: 5px; font-style: italic;">{electric_mult} × {flying_mult} = {combined_mult:.3f}</div>
        </div>
    </div>
</div>
"""

# Affichage
display(HTML(html))